# Import packages

In [149]:
import sys
import sklearn
import numpy as np
import pandas as pd
import os
import matplotlib as mpl
import matplotlib.pyplot as plt
from datetime import datetime

# Get the data

## Preparation of ratings data

In [150]:
ratings_df = pd.read_csv('data/ratings.csv')

# Drop na values
ratings_df = ratings_df.dropna()
ratings_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 63249 entries, 0 to 63249
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Programma  63249 non-null  object
 1   Zender     63249 non-null  object
 2   Datum      63249 non-null  object
 3   Start      63249 non-null  object
 4   Duur       63249 non-null  object
 5   Kijkers    63249 non-null  object
dtypes: object(6)
memory usage: 3.4+ MB


Converting the "Kijkers" column to an integer type does require some extra cleanup:

In [151]:
# Change notation of numbers (use dots as decimal separators instead of thousand separators)
ratings_df['Kijkers'] = ratings_df['Kijkers'].str.replace('.', '').str.replace(',', '.')

# Convert to float first (to handle decimal numbers). Non numeric will result in NaN
ratings_df['Kijkers'] = pd.to_numeric(ratings_df['Kijkers'], errors='coerce')

# Remove NaN values
print(f"Number of rows with non-numeric values: {ratings_df['Kijkers'].isna().sum()}")
ratings_df = ratings_df.dropna(subset=['Kijkers'])

# Then convert to integer (this will round the decimal numbers)
ratings_df['Kijkers'] = ratings_df['Kijkers'].astype(int)

Number of rows with non-numeric values: 1


Splitting "Datum" column into seperate parts: year, month, day of week

In [152]:
# Split "Datum" column in new features
ratings_df["Datum"] = pd.to_datetime(ratings_df["Datum"])
ratings_df["Jaar"] = ratings_df["Datum"].dt.year
ratings_df["Maand"] = ratings_df["Datum"].dt.month
ratings_df["Dag"] = ratings_df["Datum"].dt.day_of_week

ratings_df.head()

,Programma,Zender,Datum,Start,Duur,Kijkers,Jaar,Maand,Dag
0,HET 7 UUR-JOURNAAL,EEN,2016-10-01,19:00:05,00:31:38,721850,2016,10,5
1,FC DE KAMPIOENEN,EEN,2016-10-01,20:41:00,00:38:39,709606,2016,10,5
2,WEG ZIJN WIJ,EEN,2016-10-01,20:13:36,00:24:44,548239,2016,10,5
3,IEDEREEN BEROEMD,EEN,2016-10-01,19:38:10,00:29:01,523610,2016,10,5
4,COMEDY TOPPERS,VTM,2016-10-01,19:52:06,00:24:40,496216,2016,10,5


Some broadcasts in the data start after midnight. For example: they'll have Start columns with "24:22:37" as a value.

I don't feel like these outliers are usefull. When inspecting them, they mostly happen on new years eve or after special events like football games for example.
I believe these outliers are just representing people who didn't turn off their tv after this special event and therefore I don't find them relevant for our final data.

In [153]:
# Get all items that start after midnight
broadcasts_after_midnight = ratings_df[ratings_df["Start"].str.startswith("24:") | ratings_df["Start"].str.startswith("25:")]
broadcasts_after_midnight.head(10)

,Programma,Zender,Datum,Start,Duur,Kijkers,Jaar,Maand,Dag
1766,LOSLOPEND WILD,EEN,2016-12-31,24:22:37,00:24:34,552377,2016,12,5
1770,WAUTERS & VAN GEEL IN CONCERT,VTM,2016-12-31,24:34:48,00:52:22,369413,2016,12,5
4403,HET JOURNAAL LAAT,EEN,2017-05-13,24:46:08,00:16:10,482260,2017,5,5
5693,HET JOURNAAL LAAT,EEN,2017-07-16,24:07:25,00:16:50,239703,2017,7,6
9027,HET BESTE UIT VOOR DE LEEUWEN,EEN,2017-12-31,24:25:28,00:45:45,376711,2017,12,6
9030,CLOUSEAU DANST,VTM,2017-12-31,24:32:00,00:57:28,310513,2017,12,6
9037,STAN VAN SAMANG IN 'T GROOT,VTM,2017-12-31,25:29:51,00:30:09,201048,2017,12,6
11685,HET JOURNAAL LAAT,EEN,2018-05-12,24:51:28,00:16:29,254994,2018,5,5
12671,F1. WK. GP. OOSTENRIJK (S.),EEN,2018-07-01,24:10:40,00:22:52,131036,2018,7,6
12792,DEATH IN PARADISE,EEN,2018-07-07,24:26:31,00:52:16,85413,2018,7,5


In [154]:
# Drop these rows
ratings_df = ratings_df[~ratings_df["Start"].str.startswith("24:") & ~ratings_df["Start"].str.startswith("25:")]
ratings_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 63208 entries, 0 to 63249
Data columns (total 9 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   Programma  63208 non-null  object        
 1   Zender     63208 non-null  object        
 2   Datum      63208 non-null  datetime64[ns]
 3   Start      63208 non-null  object        
 4   Duur       63208 non-null  object        
 5   Kijkers    63208 non-null  int64         
 6   Jaar       63208 non-null  int32         
 7   Maand      63208 non-null  int32         
 8   Dag        63208 non-null  int32         
dtypes: datetime64[ns](1), int32(3), int64(1), object(4)
memory usage: 4.1+ MB


In [155]:
# Convert "Start" and "Duur" to datetime
ratings_df["Start"] = pd.to_datetime(ratings_df["Start"])
ratings_df["Duur"] = pd.to_timedelta(ratings_df["Duur"])

# Calculate end time
ratings_df["Eind"] = ratings_df["Start"] + ratings_df["Duur"]

# Extract start and end hour
ratings_df["Start"] = ratings_df["Start"].dt.hour
ratings_df["Eind"] = ratings_df["Eind"].dt.hour

# # Calculate duration in minutes
ratings_df["Duur"] = ratings_df["Duur"].dt.total_seconds() / 60
ratings_df["Duur"] = ratings_df["Duur"].astype(int)

# Add primetime columns
ratings_df["Primetime"] = (ratings_df["Start"] >= 20) & (ratings_df["Eind"] <= 22)
ratings_df["Ends_In_Primetime"] = (ratings_df["Eind"] >= 20) & (ratings_df["Eind"] <= 22)
ratings_df["Starts_In_Primetime"] = (ratings_df["Start"] >= 20) & (ratings_df["Start"] <= 22)

ratings_df.head(20)

/var/folders/5d/_v0r0r510wncd2w765n36ksw0000gn/T/ipykernel_69815/3950754292.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  ratings_df["Start"] = pd.to_datetime(ratings_df["Start"])


,Programma,Zender,Datum,Start,Duur,Kijkers,Jaar,Maand,Dag,Eind,Primetime,Ends_In_Primetime,Starts_In_Primetime
0,HET 7 UUR-JOURNAAL,EEN,2016-10-01,19,31,721850,2016,10,5,19,False,False,False
1,FC DE KAMPIOENEN,EEN,2016-10-01,20,38,709606,2016,10,5,21,True,True,True
2,WEG ZIJN WIJ,EEN,2016-10-01,20,24,548239,2016,10,5,20,True,True,True
3,IEDEREEN BEROEMD,EEN,2016-10-01,19,29,523610,2016,10,5,20,False,True,False
4,COMEDY TOPPERS,VTM,2016-10-01,19,24,496216,2016,10,5,20,False,True,False
5,THE FUGITIVE,EEN,2016-10-01,21,121,447427,2016,10,5,23,False,False,True
6,NIEUWS 19U VTM,VTM,2016-10-01,18,42,424041,2016,10,5,19,False,False,False
7,HET 1 UUR-JOURNAAL,EEN,2016-10-01,13,22,369066,2016,10,5,13,False,False,False
8,WIELRENNEN. RONDE VAN LOMBARDIJE,EEN,2016-10-01,14,175,368549,2016,10,5,17,False,False,False
9,STADION,VTM,2016-10-01,22,44,360544,2016,10,5,23,False,False,True


In [156]:
# Drop columns that have been split into new features
ratings_df = ratings_df.drop(columns=["Datum"], axis=1)

ratings_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 63208 entries, 0 to 63249
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Programma            63208 non-null  object
 1   Zender               63208 non-null  object
 2   Start                63208 non-null  int32 
 3   Duur                 63208 non-null  int64 
 4   Kijkers              63208 non-null  int64 
 5   Jaar                 63208 non-null  int32 
 6   Maand                63208 non-null  int32 
 7   Dag                  63208 non-null  int32 
 8   Eind                 63208 non-null  int32 
 9   Primetime            63208 non-null  bool  
 10  Ends_In_Primetime    63208 non-null  bool  
 11  Starts_In_Primetime  63208 non-null  bool  
dtypes: bool(3), int32(5), int64(2), object(2)
memory usage: 3.8+ MB
